In [1]:
## reset specific variables (replace regular_expression by the variables of interest)
#%reset_selective <regular_expression>

# reset all variables
%reset -f

In [2]:
## Importing libraries

from dask import dataframe as dd
from datetime import datetime, date, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from pylab import savefig
import seaborn as sns
import pandas as pd
import csv
from pymongo import MongoClient
from mongoengine import *

%matplotlib inline

In [3]:
## Creating/Connecting Mongo DB instances

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = "mongodb+srv://<jgu>:<123>@<cluster-jgu>.mongodb.net/SMARTAttributesFilter"

connect(db='SMARTAttributesFilter', alias='SMARTAttributesFilter_alias')

connect(db='FailuresAppsLocation', alias='FailuresAppsLocation_alias')

connect(db='SMARTAtt_FailuresAppsLocation', alias='SMARTAtt_FailuresAppsLocation_alias')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [4]:
## Setting document schema

class SMARTAtt(Document):
     disk_id = FloatField(required=False, default='0')
     timestamp = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     r_sectors = FloatField(required=False, default='0')
     u_errors = FloatField(required=False, default='0')
     p_failedA = FloatField(required=False, default='0')
     p_failedB = FloatField(required=False, default='0')
     e_failedA = FloatField(required=False, default='0')
     e_failedB = FloatField(required=False, default='0')
     n_b_written = FloatField(required=False, default='0')
     n_b_read = FloatField(required=False, default='0')
     meta = {'db_alias': 'SMARTAttributesFilter_alias'}

class FailuresAppsLocation(Document):
     disk_id = FloatField(required=False, default='0')
     failure_time = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     model_y = StringField(required=False, default='0')
     app = StringField(required=False, default='0')
     node_id = FloatField(required=False, default='0')
     rack_id = FloatField(required=False, default='0')
     machine_room_id = FloatField(required=False, default='0')
     meta = {'db_alias': 'FailuresAppsLocation_alias'}

class SMARTAtt_FailuresAppsLocation(Document):
     smart_att = ReferenceField(SMARTAtt)
     failures_app_location = ReferenceField(FailuresAppsLocation)
     meta = {'db_alias': 'SMARTAtt_FailuresAppsLocation_alias'}

In [ ]:
## Deleting DB content (for the case when the goal is to test the code from zero - otherwise the db will contain several replicas)

# Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects() 

# Deleting all collection
#failuresAppsLocationTeste.delete() 

In [12]:
# Closing the connection to the DB

disconnect(alias='SMARTAttributesFilter_alias')

disconnect(alias='FailuresAppsLocation_alias_alias')

disconnect(alias='SMARTAtt_FailuresAppsLocation_alias')

In [5]:
## Loading datasaet - AlibabaOverTime (Failurelogs)

df_AlibabaOver_Failurelogs = pd.read_csv('/media/erb/hdd1/DataSet/alibabaOvertime/ssd_failure_label/ssd_failure_label.csv')

In [6]:
## Loading dataset - Alibaba Snapshot (TimeStamps of failed SSDs, SMART attributes in 39 columns, SSDs location, applications, SSD models and Disk ID)

df_AlibabaSnapShot_FailuresAppsLocation = pd.read_csv('/media/erb/hdd1/DataSet/alibabaSnapShot/ssd_failure_tag/ssd_failure_tag.csv')

In [7]:
# Merging Failures and location datasets and fixing columns types (to have ssd failure data that has location, failure time, and smart att)
df_Failurelogs_FailuresAppsLocation =  pd.merge(df_AlibabaOver_Failurelogs, df_AlibabaSnapShot_FailuresAppsLocation, how = 'inner', on = ['disk_id', 'failure_time'])

# Changing failure time column to datetime type
df_Failurelogs_FailuresAppsLocation['failure_time'] =  pd.to_datetime(df_Failurelogs_FailuresAppsLocation['failure_time'])

# Removing duplicates
#df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.drop_duplicates(subset='disk_id', keep="first")

# Forcing sorting
df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.sort_values(by=['failure_time'], ascending=True)

# Choosing the columns of interest
df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.loc[:,['disk_id','failure_time', 'model_x','model_y','app','node_id','rack_id','machine_room_id']]

# Changing data type
#df_Failurelogs_FailuresAppsLocation = df_Failurelogs_FailuresAppsLocation.astype(datatype)

# Testing
#df_Failurelogs_FailuresAppsLocation.head(20)
#df_Failurelogs_FailuresAppsLocation.query(('rack_id == 17596 & app == "RM"'))
#df_Failurelogs_FailuresAppsLocation.query(('app == "RM"'))
#time1 = df_Failurelogs_FailuresAppsLocation.query(('rack_id == 17596 & app == "RM" & disk_id==39876'))
#time2 = df_Failurelogs_FailuresAppsLocation.query(('rack_id == 17596 & app == "RM" & disk_id==22968'))

In [8]:
## Inserting FailuresAppsLocation into the DB

for row in df_Failurelogs_FailuresAppsLocation.itertuples():
    insert_FailuresAppsLocation = FailuresAppsLocation()
    insert_FailuresAppsLocation.disk_id = row.disk_id
    insert_FailuresAppsLocation.failure_time = row.failure_time
    insert_FailuresAppsLocation.model_x = row.model_x
    insert_FailuresAppsLocation.model_y = row.model_y
    insert_FailuresAppsLocation.app = row.app
    insert_FailuresAppsLocation.node_id = row.node_id
    insert_FailuresAppsLocation.rack_id = row.rack_id
    insert_FailuresAppsLocation.machine_room_id = row.machine_room_id
    insert_FailuresAppsLocation.save()

In [11]:
## Some functions in Mongoengine

# Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects()  

# Query if you know something about the document
#testando = FailuresAppsLocation.objects(disk_id="33722").get()

# Printing
#testando.disk_id

# Deleting all collection
#failuresAppsLocationTeste.delete()

In [25]:
# Loading AlibabaOvertime dataset using Pandas

start_date = date(2018, 4, 22)
end_date = date(2019, 12, 31)
delta = timedelta(days=1)
df_AlibabaOver_SMARTlogs = pd.DataFrame()


while start_date <= end_date:
    path = Path('/media/erb/hdd1/DataSet/alibabaOvertime/smartlogs/' + start_date.strftime("%Y%m%d") + '.csv')

    if path.is_file(): # checking if a particular file for a specific date is missing
        df_AlibabaOver_SMARTlogs = pd.read_csv(path)
        df_AlibabaOver_SMARTlogs = pd.DataFrame(df_AlibabaOver_SMARTlogs)

        # Changing failure time column to datetime type
        df_AlibabaOver_SMARTlogs['ds'] =  pd.to_datetime(df_AlibabaOver_SMARTlogs['ds'], format='%Y%m%d')

        # Choosing the columns of interest
        df_AlibabaOver_SMARTlogs = df_AlibabaOver_SMARTlogs.loc[:,['disk_id','ds', 'model','n_5','n_187','n_171','n_181','n_172','n_182','n_241','n_242']]

        # Changing the name of some columns to clarify their meaning
        df_AlibabaOver_SMARTlogs.rename(columns = {'ds':'timestamp', 'model':'model_x', 'n_5':'r_sectors','n_187':'u_errors','n_171':'p_failedA','n_181':'p_failedB','n_172':'e_failedA','n_182':'e_failedB','n_241':'n_b_written','n_242':'n_b_read'}, inplace=True)

        for row in df_AlibabaOver_SMARTlogs.itertuples():
            insert_SmartAttributes = SMARTAtt()
            insert_SmartAttributes.disk_id = row.disk_id
            insert_SmartAttributes.timestamp = row.timestamp
            insert_SmartAttributes.model_x = row.model_x

            #Checking if the value is int/float. Without this checking an error may be raised during mongoengine validation (saving)

            if isinstance(row.r_sectors, (int, float)):
                insert_SmartAttributes.r_sectors = row.r_sectors
            else: 
                insert_SmartAttributes.r_sectors = 0
            if isinstance(row.u_errors, (int, float)):
                insert_SmartAttributes.u_errors = row.u_errors
            else: 
                insert_SmartAttributes.u_errors = 0
            if isinstance(row.p_failedA, (int, float)):
                insert_SmartAttributes.p_failedA = row.p_failedA
            else: 
                insert_SmartAttributes.p_failedA = 0
            if isinstance(row.p_failedB, (int, float)):
                insert_SmartAttributes.p_failedB = row.p_failedB
            else: 
                insert_SmartAttributes.p_failedB = 0
            if isinstance(row.e_failedA, (int, float)):
                insert_SmartAttributes.e_failedA = row.e_failedA
            else: 
                insert_SmartAttributes.e_failedA = 0
            if isinstance(row.e_failedB, (int, float)):
                insert_SmartAttributes.e_failedB = row.e_failedB
            else: 
                insert_SmartAttributes.e_failedB = 0
            if isinstance(row.n_b_written, (int, float)):
                insert_SmartAttributes.n_b_written = row.n_b_written
            else: 
                insert_SmartAttributes.n_b_written = 0
            if isinstance(row.n_b_read, (int, float)):
                insert_SmartAttributes.n_b_read = row.n_b_read
            else: 
                insert_SmartAttributes.n_b_read = 0
            insert_SmartAttributes.save()

        #df_AlibabaOver_SMARTlogs_Filtered = pd.concat([df_AlibabaOver_SMARTlogs_Filtered, df_AlibabaOver_SMARTlogs], ignore_index=True)
    start_date += delta

# Changing the name of some columns to clarify their meaning
#df_AlibabaOver_SMARTlogs_Filtered.rename(columns = {'ds':'timestamp', 'model':'model_x', 'n_5':'r_sectors','n_187':'u_errors','n_171':'p_failedA','n_181':'p_failedB','n_172':'e_failedA','n_182':'e_failedB','n_241':'n_b_written','n_242':'n_b_read'}, inplace=True)

In [ ]:
# Loading AlibabaOvertime dataset using Dask

start_date = date(2018, 2, 1)
start_dateAux = start_date
end_date = date(2018, 5, 31)
delta = timedelta(days=1)

while start_date <= end_date:
    path = Path('/media/erb/hdd1/DataSet/alibabaOvertime/smartlogs/' + start_date.strftime("%Y%m%d") + '.csv')

    if path.is_file(): # checking if a particular file for a specific date is missing
        df_AlibabaOver_SMARTlogsTemp = dd.read_csv(path)
        if start_date == start_dateAux:
            df_AlibabaOver_SMARTlogs = df_AlibabaOver_SMARTlogsTemp
        else: 
            df_AlibabaOver_SMARTlogs = dd.concat([df_AlibabaOver_SMARTlogs, df_AlibabaOver_SMARTlogsTemp])
    start_date += delta

In [37]:
# Testing Alibaba's SMART attributes over two years dataset 

# settings to display all columns
#pd.set_option("display.max_columns", None)

#df_AlibabaOver_SMARTlogs_Filtered.query('disk_id == 4711')
#df_AlibabaOver_Failurelogs.head(10)
#df_Failurelogs_FailuresAppsLocation.dtypes

# Creating the object related to the whole collection
#SMARTAttributesTest = SMARTAtt.objects()  

# Query if you know something about the document
SMARTAttributesTest = SMARTAtt.objects(timestamp="2018-04-22").filter()


# Printing
for i in SMARTAttributesTest:
  print(i.timestamp)
  
# Deleting all collection
#for i in SMARTAttributesTest:
 # i.delete()


In [ ]:
# Loading Backblaze dataset

start_dateBB = date(2021, 12, 29)
start_dateAuxBB = start_dateBB
end_dateBB = date(2021, 12, 31)
deltaBB = timedelta(days=1)

while start_dateBB <= end_dateBB:
    pathBB = Path('/media/erb/hdd1/DataSet/backblaze/smartlogs/' + start_dateBB.strftime("%Y-%m-%d") + '.csv')

    if pathBB.is_file(): # checking if a particular file for a specific date is missing
        df_BackBlaze_SMARTlogsTemp = pd.read_csv(pathBB)
        if start_dateBB == start_dateAuxBB: # due to dask instancing of variables
            df_BackBlaze_SMARTlogs = df_BackBlaze_SMARTlogsTemp
        else: 
            df_BackBlaze_SMARTlogs = pd.concat([df_BackBlaze_SMARTlogs, df_BackBlaze_SMARTlogsTemp])
    start_dateBB += deltaBB  

In [ ]:
# Testing BackBlaze dataset

df_BackBlaze_SMARTlogs.head()

In [ ]:
# Carregando dados filtrados em um dataframe para graficos de performance

dataset = pd.read_csv("dataset.csv")

# Separando por cenários

##Cenário 1
dataset_cenUm = dataset.query('scenario == 1')

### Escrita
dataset_write_cenUm = dataset_cenUm.query('operation == "W" & individual_total == "T"')



In [ ]:
## RETIRANDO RUÍDOS DAS MEDIÇÕES DE TENSÃO para performance

##Cenário 1

### Escrita

dataset_write_cenUm.power_average1_hdd = dataset_write_cenUm.power_average1_hdd.astype(str)
dataset_write_cenUm['power_average1_hdd'] = dataset_write_cenUm['power_average1_hdd'].apply(lambda x: x.split("e")[0])
dataset_write_cenUm.power_average1_hdd = dataset_write_cenUm.power_average1_hdd.astype(float)

In [ ]:
# CENARIO 1

## WRITE IOPS

fig = sns.catplot(x="capacity", y="iops_total", hue="object_size", data=dataset_write_cenUm,
               row="pattern", col="work_outstd", kind="bar", ci=90, palette="Blues_d", aspect=0.9, height=4.5
                  , legend_out = True, margin_titles = True)

fig.set_axis_labels("Capacity", "IOPS")
fig.set_xticklabels(["80GB HDD", "500GB HDD", "1TB HDD", "1TB WDHDD", "120GB SSD", "Hybrid"])

plt.savefig('write_iops.pdf', dpi=1200)